### **직렬화(Serialization)**

- 모델을 저장 가능한 형식으로 변환하는 과정

#### **목적**
- 모델 재사용 (재훈련 없이)
- 모델 배포 및 공유 용이
- 계산 리소스 절약

#### **장점**
- 빠른 모델 로딩
- 버전 관리 가능
- 다양한 환경에서 사용 가능
- 모델 직렬화는 AI 개발 및 배포 과정에서 중요한 단계로, 효율적인 모델 관리와 재사용을 가능하게 합니다.

In [16]:
import os
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")

# 직렬화가 여부 확인
print(f"ChatGroq: {ChatGroq.is_lc_serializable()}")

llm = ChatGroq(model = 'gemma2-9b-it')

chain = prompt | llm

# 체인 직렬화 여부 확인
chain.is_lc_serializable()

ChatGroq: True


True

### **Chain 직렬화(dumps, dumpd)**

- 직렬화 가능한 객체를 Dict Or JSON 형식으로 저장 

#### **직렬화 방법**
객체의 속성 및 데이터를 키-값 쌍으로 저장하여 딕셔너리 형태로 변환

이러한 직렬화 방식은 객체를 쉽게 저장하고 전송할 수 있게 하며, 다양한 환경에서 객체를 재구성할 수 있도록 함


`dumpd`: 객체를 딕셔너리로 직렬화


`dumps`: 객체를 JSON 문자열로 직렬화

In [19]:
from langchain_core.load import dumpd

dumpd_chain = dumpd(chain)

print(dumpd_chain)
type(dumpd_chain)

{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'], 'kwargs': {'first': {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'], 'kwargs': {'input_variables': ['fruit'], 'template': '{fruit}의 색상이 무엇입니까?', 'template_format': 'f-string'}, 'name': 'PromptTemplate'}, 'last': {'lc': 1, 'type': 'constructor', 'id': ['langchain_groq', 'chat_models', 'ChatGroq'], 'kwargs': {'model_name': 'gemma2-9b-it', 'temperature': 0.7, 'groq_api_key': {'lc': 1, 'type': 'secret', 'id': ['GROQ_API_KEY']}, 'max_retries': 2, 'n': 1}, 'name': 'ChatGroq'}}, 'name': 'RunnableSequence'}


dict

In [21]:
from langchain_core.load import dumps

dumps_chain = dumps(chain)

print(dumps_chain)
type(dumps_chain)

{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["fruit"], "template": "{fruit}\uc758 \uc0c9\uc0c1\uc774 \ubb34\uc5c7\uc785\ub2c8\uae4c?", "template_format": "f-string"}, "name": "PromptTemplate"}, "last": {"lc": 1, "type": "constructor", "id": ["langchain_groq", "chat_models", "ChatGroq"], "kwargs": {"model_name": "gemma2-9b-it", "temperature": 0.7, "groq_api_key": {"lc": 1, "type": "secret", "id": ["GROQ_API_KEY"]}, "max_retries": 2, "n": 1}, "name": "ChatGroq"}}, "name": "RunnableSequence"}


str

### **Save and Load**

#### **Pickle**

Pickle 파일은 Python 객체를 바이너리 형태로 직렬화하는 포맷

- Python 전용 (다른 언어와 호환 불가)
- 대부분의 Python 데이터 타입 지원 (리스트, 딕셔너리, 클래스 등)
- 객체의 상태와 구조를 그대로 보존

```bash
pickle.dump(): 객체를 파일에 저장
pickle.load(): 파일에서 객체 로드
```

In [ ]:
import pickle

# pickle로 저장  바이너리 모드이기 떄문에(wb)
with open('pickle/chain.pkl', 'wb') as f:
    pickle.dump(dumpd_chain, f)

# pickle 불러오기
with open('pickle/chain.pkl', 'rb') as f:
    pickle_chain = pickle.load(f)
    
pickle_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain_groq', 'chat_models', 'ChatGroq'],
   'kwargs': {'model_name': 'gemma2-9b-it',
    'temperature': 0.7,
    'groq_api_key': {'lc': 1, 'type': 'secret', 'id': ['GROQ_API_KEY']},
    'max_retries': 2,
    'n': 1},
   'name': 'ChatGroq'}},
 'name': 'RunnableSequence'}

In [ ]:
# 직렬화된 체인을 다시 로드
from langchain_core.load import load

# pickle 불러오기
with open('pickle/chain.pkl', 'rb') as f:
    pickle_chain = pickle.load(f)
    
chain_loaded = load(pickle_chain)
response = chain_loaded.invoke('사과')
print(response.content)

사과는 일반적으로 **빨강** , **초록** , **노랑** 색상으로 주로 볼 수 있습니다. 

하지만 품종에 따라 다양한 색상을 가질 수 있습니다. 예를 들어, 

* **빨간색 사과**:  포도색 사과, 벌플 사과
* **초록색 사과**: 그린 사과
* **노란색 사과**: 골든 딜리시어스 사과

등이 있습니다.  



In [43]:
import json

# json으로 저장
with open('pickle/chain.json', 'w') as f:
    json.dump(dumpd_chain, f)

# json 불러오기
with open('pickle/chain.json', 'r') as f:
    json_chain = json.load(f)
    # langchain_core.load.load 함수로 로드
    chain_loaded_json = load(json_chain)
    
response = chain_loaded_json.invoke('사과')
print(response.content)

사과의 색상은 다양합니다. 🍎

가장 흔한 색상은 **빨강**, **초록색**, **노랑색**입니다. 

하지만, 그 외에도 **주황색**, **연분홍색**, ** PURPLE** 등의 색상을 가진 사과도 있습니다. 

어떤 사과의 색상이 가장 좋다고 생각하시나요? 🤔

